## Web Scraping of the Box Offices Mojo Database provided from IMdB

https://www.boxofficemojo.com/year/world/2019/

Import the libraries:

In [1]:
import pandas as pd
import numpy as np
import re
import datetime
import requests
import bs4

pd.set_option('display.max_rows', 50)


Base and variable URL:

In [2]:
base2 = "https://www.boxofficemojo.com/year/world/"
base = "https://www.boxofficemojo.com/year/"
sufix = "/?grossesOption=totalGrosses"
years = [str(i)+"/" for i in range(2000,2020)]

Empty DataFrame:

In [3]:
df = pd.DataFrame(columns=["year","release", "gross" , "max_th", "opening", "open_th", "open", "close", "distributor"])


Scrap the data for the last 20 years (2018 - 2019):

In [4]:
# problematic stucture when there is no distributor, small function to solve it
def distributor(movies):
    if movies.find_all("td", class_="a-text-left mojo-field-type-studio")[0].find("a", class_="a-link-normal") == None:
        return("-")
    else:
        return movies.find_all("td", class_="a-text-left mojo-field-type-studio")[0].find("a", class_="a-link-normal").contents[0] 


In [5]:
# empty dataframe
df = pd.DataFrame(columns=["year","rank","release", "gross" , "max_th", "opening", "open_th", "open", "close", "distributor"])

# loop for fetch the data
for year in years:
    url = base + year + sufix
    r = requests.get(url)
    soup = bs4.BeautifulSoup(r.content, "html.parser")
    movies = soup.find("div", class_="a-section imdb-scroll-table-inner").find_all("tr")
    for movie in movies[1:]:
        rank = str(movie.find_all("td", class_="a-text-right mojo-header-column mojo-truncate mojo-field-type-rank mojo-sort-column")[0].contents[0])
        release = str(movie.find("a", class_="a-link-normal").contents[0])
        gross = int(str(movie.find_all("td", class_="a-text-right mojo-field-type-money mojo-estimatable")[0].contents[0].replace(",","").replace("$","").replace("-","0")))
        max_th = int(str(movie.find_all("td", class_="a-text-right mojo-field-type-positive_integer")[0].contents[0].replace(",","").replace("-","0")))
        opening = int(str(movie.find_all("td", class_="a-text-right mojo-field-type-money")[0].contents[0].replace(",","").replace("$","").replace("-","0")))
        open_th = int(str(movie.find_all("td", class_="a-text-right mojo-field-type-positive_integer")[1].contents[0].replace(",","").replace("-","0")))
        open_ = str(movie.find_all("td", class_="a-text-left mojo-field-type-date a-nowrap")[0].contents[0])
        close_ = str(movie.find_all("td", class_="a-text-left mojo-field-type-date a-nowrap")[1].contents[0])
        distributor = str("-" if movie.find_all("td", class_="a-text-left mojo-field-type-studio")[0].find("a", class_="a-link-normal") == None else movie.find_all("td", class_="a-text-left mojo-field-type-studio")[0].find("a", class_="a-link-normal").contents[0])
                                            
        df = df.append({"year":int(year[:-1]), "rank":rank, "release":release, "gross":gross, "max_th":max_th, "opening":opening, "open_th":open_th, "open":open_, "close":close_, "distributor":distributor}, ignore_index=True)
     

In [6]:
df

,year,rank,release,gross,max_th,opening,open_th,open,close,distributor
0,2000,1,How the Grinch Stole Christmas,260044825,3256,55082330,3127,Nov 17,Mar 1,Universal Pictures
1,2000,2,Cast Away,233632142,3061,28883406,2774,Dec 22,-,Twentieth Century Fox
2,2000,3,Mission: Impossible II,215409889,3669,57845297,3653,May 24,Oct 19,Paramount Pictures
3,2000,4,Gladiator,187705427,3188,34819017,2938,May 5,May 10,DreamWorks Distribution
4,2000,5,What Women Want,182811707,3092,33614543,3012,Dec 15,-,Paramount Pictures
...,...,...,...,...,...,...,...,...,...,...
12229,2019,786,Game Day,1624,4,1624,4,Oct 4,-,Ammo Content
12230,2019,787,American Playhouse: The Killing Floor,1583,1,1583,1,Nov 22,-,Film Movement
12231,2019,788,The Hours and Times,1273,1,1273,1,Mar 1,-,Oscilloscope
12232,2019,789,The Untold Story,790,1,790,1,Jan 18,-,Ammo Content


It works! :)

In [7]:
# df.to_csv("us_bo.csv")